In [12]:
rm(list=ls())
suppressPackageStartupMessages(require(skimr))
suppressPackageStartupMessages(require(readxl))
suppressPackageStartupMessages(require(stringr))
suppressPackageStartupMessages(require(dplyr))
suppressPackageStartupMessages(require(ggplot2))
suppressPackageStartupMessages(require(forecast))

In [13]:
#------------------------------------------------------
# data pre treatment (similar to univariate case)

df <- read_excel("data/data.xlsx",sheet = 3)
tmp <-df%>% filter(aire=="44_GRAND EST" & champ=="ESS")

df <-tmp[-which(!is.na(tmp$secret_stat)),]%>%select(-secret_stat)
df <- df %>% select(-champ,-type_aire,-type_naf,-aire)
df <- df[,c(TRUE,str_detect(colnames(df),"nb_etab")[-1])]
tmp <- t(df)
colnames(tmp) <- tmp[1,]
tmp <-tmp[-1,]
df <- tmp
tmp <-df%>%as.data.frame
tmp <-cbind("date"=rep(NA,nrow(tmp)),tmp)
for (i in 1:nrow(tmp)){
    tmp.format.date <- strsplit(rownames(df)[i],"nb_etab")[[1]][2]
    subsplit <-strsplit(tmp.format.date,"T")[[1]]
    year<-subsplit[1]
    trimester <- subsplit[2]
    month <-switch(trimester,
        "1"="01",
        "2"="04",
        "3"="07",
        "4"="10"
    )
    tmp[i,1] <- paste(year,month,"01",sep="-")
}
tmp$date <-as.Date(tmp$date)
rownames(tmp) <- NULL

df.indexes <-which(tmp$date <= as.Date("2020-01-01"))
train.indexes<- which(tmp$date < as.Date("2018-01-01") )
test.indexes <-which(tmp$date>= as.Date("2018-01-01") & tmp$date <as.Date("2020-01-01") )
tmp <-tmp %>%select(-date)%>% mutate_all(as.numeric)
#----------------------------
# Eliminate total serie
tmp <- tmp[,-ncol(tmp)]
df.ts <- ts(tmp[df.indexes,],start=c(2010,1),frequency=4)# S'arrete à 2020 Q1 
train.ts <-ts(tmp[train.indexes,],start=c(2010,1),frequency = 4)# S'arrete à 2017 Q4
test.ts <-ts(tmp[test.indexes,],start = c(2018,1),frequency = 4)# Commence a 2018 Q1 --> 2019 Q4
global.ts <- ts(tmp,start=c(2010,1),frequency=4)# Commence a 2010 Q1 --> 2021 Q2
covid.ts <- ts(tmp[(df.indexes[length(df.indexes)]+1):nrow(tmp),],start=c(2020,2),frequency = 4)# Commence a 2020 Q2 --> 2021 Q2

# Univariate predictions

In [197]:
calculate_RMSE <- function(predictions,obs){
    return(sqrt(mean((obs-pred)^2)))
}

In [198]:
univariate_RMSE <- c()
for(i in 1:26){
    tmp <-readRDS(paste("plots/univariate/models/",i,".Rda",sep=""))
    fit.val = df.ts[,i]- tmp$residuals
    new <-fit.val[1:(length(fit.val)-5)]
    new <-new %>% ts(start=c(2010,1),frequency=4)
    new_mod<-Arima(new,model=tmp)
    pred <-forecast(new_mod,h=5)$mean
    obs <- df.ts[(nrow(df.ts)-4):nrow(df.ts),i]
    
    
    pred <-pred %>%as.numeric
    

    univariate_RMSE <-c(univariate_RMSE,calculate_RMSE(pred,obs))
}


[1] 1.920034

In [199]:
univariate_RMSE %>%as.data.frame

.
<dbl>
16.935856
8.034724
2.437679
1.483240
26.993462
1.165867
2.013560
2.489980
3.711552


<hr>

## Multivariate predictions


### A. VAR /VECM

In [200]:
mod.eval.df <- readRDS("VAR_VECM_models/df_eval_mod_VAR.Rda")


In [201]:

VAR_mod_RMSE <- c()
for(i in 1:26){
    tmp <- readRDS(paste("VAR_VECM_models/models/",i,".Rda",sep=""))
    errors <-residuals(tmp)
    row_eval <-mod.eval.df%>%filter(VAR==i)
    combi <-row_eval$MOD_COMBINATION
    numbers.combi <-strsplit(combi,"[_]")[[1]]
    for (j in 1:(numbers.combi%>% length)){


        if(as.numeric(numbers.combi[j])==i){
            break;
        }

    }
    if(row_eval$adequate!="adequateVECM"){
        pred <- fitted(tmp)[,j]
        pred <- diffinv(pred,xi=df.ts[1,i])
    
    
    
    }else{
        #Pas besoin de differencier
        pred <- fitted(tmp)[,j]
    }
    obs <- df.ts[1:length(pred),i]
    pred<-pred[(length(pred)-4):length(pred)] 
    obs<- obs[(length(obs)-4):length(obs)]
    
    VAR_mod_RMSE <- c(VAR_mod_RMSE,calculate_RMSE(pred,obs))
    
    
    
    
}
VAR_mod_RMSE %>% as.data.frame

.
<dbl>
28.3302640
7.1191478
1.4947224
1.0990141
33.2512588
2.8232795
7.6546313
5.3702190
18.7602090


In [202]:
RMSE_res <-cbind(VAR_mod_RMSE,univariate_RMSE)
RMSE_res

VAR_mod_RMSE,univariate_RMSE
28.3302640,16.935856
7.1191478,8.034724
1.4947224,2.437679
1.0990141,1.483240
33.2512588,26.993462
2.8232795,1.165867
7.6546313,2.013560
5.3702190,2.489980
18.7602090,3.711552
4.0227753,4.335897


In [203]:
sum((VAR_mod_RMSE < univariate_RMSE)) 
#Pour 6 variables on a de meilleurs prédictions avec le multivarié
# Ce sont des variables ayant de bonne correlation s avec le reste
# Elles ne sont pas les plus correllées aux autres ni les plus decorellées

[1] 6

In [204]:
which((VAR_mod_RMSE < univariate_RMSE))

[1]  2  3  4 10 11 24

### B. LASSO REGRESSION

In [155]:
require(bigtime)

Loading required package: bigtime



In [206]:
LASSO_mod_RMSE <- c()
df.scaled <- df.ts %>% as.data.frame()%>%scale

for(i in 1:26){
    centered_scale <-attr(df.scaled,"scaled:center")[i]
    sd_scale <-attr(df.scaled,"scaled:scale")[i]
    pred <-diagnostics_plot(tmp,variable=paste("DF",i,sep=""))$data %>% filter(Series=="Fitted" )
    pred <- pred$value
    obs <- df.ts[,i]

    obs <- obs[
        (length(obs)-length(pred)+1):length(obs)
        ]

    c <-rep(centered_scale[[1]],length(pred))
    scale <-rep(sd_scale[[1]],length(pred))
    pred <-pred*scale +c
    pred <- pred[(length(pred)-4):length(pred)]
    obs <- obs[(length(obs)-4):length(obs)]

    LASSO_mod_RMSE <- c(LASSO_mod_RMSE,calculate_RMSE(pred,obs))
}
LASSO_mod_RMSE %>% as.data.frame




.
<dbl>
6.3307653
4.7584142
0.6701038
0.3679150
5.7198987
0.7333771
1.7671146
1.5683655
3.1402716


In [207]:
RMSE_res <-cbind(VAR_mod_RMSE,univariate_RMSE,LASSO_mod_RMSE)
RMSE_res

VAR_mod_RMSE,univariate_RMSE,LASSO_mod_RMSE
28.3302640,16.935856,6.3307653
7.1191478,8.034724,4.7584142
1.4947224,2.437679,0.6701038
1.0990141,1.483240,0.3679150
33.2512588,26.993462,5.7198987
2.8232795,1.165867,0.7333771
7.6546313,2.013560,1.7671146
5.3702190,2.489980,1.5683655
18.7602090,3.711552,3.1402716
4.0227753,4.335897,2.6944138


### C. LSTM 

In [212]:
RMSE_LSTM_stable <-c(
2.35594041,
4.90520036,	
2.04205399,	
0.17883123,	
23.37337711,
3.36964486,	
5.92676943,	
2.47419939,	
3.17811455,
3.10329963,
4.88695701,
12.47542726,
10.59384383,	
1.61290141,	
11.66858881,	
0.03346412,
6.94559225,
2.77303550,	
1.53613439,
1.09849246,	
1.35458494,	
0.86600634,	
4.08833856,	
10.88680699,
37.24070985,	
2.47525374
)

# SELECTION DES MODELES

In [214]:
RMSE_res <- cbind(RMSE_res,RMSE_LSTM_stable)

In [215]:
RMSE_res

VAR_mod_RMSE,univariate_RMSE,LASSO_mod_RMSE,RMSE_LSTM_stable
28.3302640,16.935856,6.3307653,2.35594041
7.1191478,8.034724,4.7584142,4.90520036
1.4947224,2.437679,0.6701038,2.04205399
1.0990141,1.483240,0.3679150,0.17883123
33.2512588,26.993462,5.7198987,23.37337711
2.8232795,1.165867,0.7333771,3.36964486
7.6546313,2.013560,1.7671146,5.92676943
5.3702190,2.489980,1.5683655,2.47419939
18.7602090,3.711552,3.1402716,3.17811455
4.0227753,4.335897,2.6944138,3.10329963


In [216]:
#Already done
apply(RMSE_res,1,function(x){
    lab.ind <-which.min(x)
    return(colnames(RMSE_res)[lab.ind])
})

[1] "RMSE_LSTM_stable" "LASSO_mod_RMSE"   "LASSO_mod_RMSE"   "RMSE_LSTM_stable"
 [5] "LASSO_mod_RMSE"   "LASSO_mod_RMSE"   "LASSO_mod_RMSE"   "LASSO_mod_RMSE"  
 [9] "LASSO_mod_RMSE"   "LASSO_mod_RMSE"   "LASSO_mod_RMSE"   "LASSO_mod_RMSE"  
[13] "LASSO_mod_RMSE"   "LASSO_mod_RMSE"   "LASSO_mod_RMSE"   "univariate_RMSE" 
[17] "LASSO_mod_RMSE"   "LASSO_mod_RMSE"   "LASSO_mod_RMSE"   "LASSO_mod_RMSE"  
[21] "LASSO_mod_RMSE"   "RMSE_LSTM_stable" "LASSO_mod_RMSE"   "RMSE_LSTM_stable"
[25] "LASSO_mod_RMSE"   "LASSO_mod_RMSE"

In [ ]:
# => ON en conclut selon ce critere que l'on va analyser les ecart à l'aide du modèles LSTM